In [19]:
import pandas as pd

df_mnq = pd.read_csv("../futures/NQ/20220220-20220304-1.csv")
df_mnq = df_mnq[["date", "close"]]
df_mym = pd.read_csv("../futures/YM/20220220-20220304-1.csv")
df_mym = df_mym[["date", "close"]]

In [20]:

df = pd.merge(df_mnq, df_mym, on="date", suffixes=("_mnq", "_mym"))
df.index = pd.to_datetime(df["date"])
df.drop(columns=["date"], inplace=True)
df

,close_mnq,close_mym
date,,
2022-02-20 17:00:00,13932.75,33921.0
2022-02-20 17:01:00,13859.50,33907.0
2022-02-20 17:02:00,13884.75,33923.0
2022-02-20 17:03:00,13875.50,33907.0
2022-02-20 17:04:00,13873.50,33915.0
...,...,...
2022-03-04 15:55:00,13800.50,33524.0
2022-03-04 15:56:00,13802.00,33524.0
2022-03-04 15:57:00,13800.00,33526.0


In [21]:
import numpy as np
import talib
from talib import MA_Type

df["px_diff"] = np.log(df["close_mym"].divide(df["close_mnq"]))
upper, mid, lower = talib.BBANDS(df["px_diff"], timeperiod=10, nbdevup=2, nbdevdn=2, matype=MA_Type.SMA)
df["px_diff_upper"] = upper
df["px_diff_mid"] = mid
df["px_diff_lower"] = lower
df

,close_mnq,close_mym,px_diff,px_diff_upper,px_diff_mid,px_diff_lower
date,,,,,,
2022-02-20 17:00:00,13932.75,33921.0,0.889792,NaN,NaN,NaN
2022-02-20 17:01:00,13859.50,33907.0,0.894651,NaN,NaN,NaN
2022-02-20 17:02:00,13884.75,33923.0,0.893302,NaN,NaN,NaN
2022-02-20 17:03:00,13875.50,33907.0,0.893497,NaN,NaN,NaN
2022-02-20 17:04:00,13873.50,33915.0,0.893877,NaN,NaN,NaN
...,...,...,...,...,...,...
2022-03-04 15:55:00,13800.50,33524.0,0.887557,0.888013,0.887704,0.887394
2022-03-04 15:56:00,13802.00,33524.0,0.887448,0.887962,0.887656,0.887350
2022-03-04 15:57:00,13800.00,33526.0,0.887653,0.887824,0.887621,0.887419


In [22]:
df_selected = df.between_time("02:30", "06:30")
df_selected

,close_mnq,close_mym,px_diff,px_diff_upper,px_diff_mid,px_diff_lower
date,,,,,,
2022-02-21 02:30:00,14095.25,34268.0,0.888374,0.888567,0.888314,0.888061
2022-02-21 02:31:00,14097.75,34278.0,0.888489,0.888607,0.888335,0.888063
2022-02-21 02:32:00,14105.25,34287.0,0.888219,0.888608,0.888333,0.888058
2022-02-21 02:33:00,14097.00,34277.0,0.888513,0.888634,0.888369,0.888104
2022-02-21 02:34:00,14095.25,34275.0,0.888578,0.888685,0.888395,0.888106
...,...,...,...,...,...,...
2022-03-04 06:26:00,13893.75,33387.0,0.876728,0.877050,0.876695,0.876341
2022-03-04 06:27:00,13891.25,33383.0,0.876788,0.877061,0.876703,0.876344
2022-03-04 06:28:00,13877.75,33361.0,0.877101,0.877156,0.876729,0.876303


In [23]:
from datetime import time
from pandas import DataFrame

executions = []
current_pnl = 0

MNQ_LOT = 1
MYM_LOT = 3

for dt, row in df_selected.iterrows():
    last = executions[-1] if executions else None

    # Exit - no cross-day position
    if dt.time() == time(6, 30):
        executions[-1] |= {
            "exit_mnq": row["close_mnq"],
            "exit_mym": row["close_mym"],
            "exit_t": dt,
            "position": "close",
        }

    # Entry - out of band
    if not last or last["position"] == "close":
        if row["px_diff"] > row["px_diff_upper"]:
            executions.append({
                "buy": "MNQ",
                "sell": "MYM",
                "entry_t": dt,
                "entry_mnq": row["close_mnq"],
                "entry_mym": row["close_mym"],
                "exit_mnq": None,
                "exit_mym": None,
                "exit_t": None,
                "position": "open",
                "pnl_highest": 0,
                "pnl_lowest": 0,
            })

        if row["px_diff"] < row["px_diff_lower"]:
            executions.append({
                "buy": "MYM",
                "sell": "MNQ",
                "entry_t": dt,
                "entry_mnq": row["close_mnq"],
                "entry_mym": row["close_mym"],
                "exit_mnq": None,
                "exit_mym": None,
                "exit_t": None,
                "position": "open",
                "pnl_highest": 0,
                "pnl_lowest": 0,
            })

        continue
    # Take profit - back to MID
    elif (
            (last["buy"] == "MNQ" and row["px_diff"] < row["px_diff_mid"]) or
            (last["buy"] == "MYM" and row["px_diff"] > row["px_diff_mid"])
    ):
        executions[-1] |= {
            "exit_mnq": row["close_mnq"],
            "exit_mym": row["close_mym"],
            "exit_t": dt,
            "position": "close",
        }
        current_pnl = 0

    # Record PnL
    if last["position"] == "open":
        pnl_mnq = (row["close_mnq"] - last["entry_mnq"]) * (1 if last["buy"] == "MNQ" else -1) * MNQ_LOT * 2
        pnl_mym = (row["close_mym"] - last["entry_mym"]) * (1 if last["buy"] == "MYM" else -1) * MYM_LOT * 0.5

        current_pnl = pnl_mnq + pnl_mym

        executions[-1] |= {
            "pnl_highest": max(last["pnl_highest"], current_pnl),
            "pnl_lowest": min(last["pnl_lowest"], current_pnl),
        }

    # Take profit - lock profit
    if current_pnl < 10 and last["pnl_highest"] > 40:
        executions[-1] |= {
            "exit_mnq": row["close_mnq"],
            "exit_mym": row["close_mym"],
            "exit_t": dt,
            "position": "close",
        }

    # Stop loss - force close
    if current_pnl < -50:
        executions[-1] |= {
            "exit_mnq": row["close_mnq"],
            "exit_mym": row["close_mym"],
            "exit_t": dt,
            "position": "close",
        }

df_exec = DataFrame(executions)
df_exec["hold_period"] = df_exec["exit_t"] - df_exec["entry_t"]
df_exec["px_side_mnq"] = (df_exec["exit_mnq"] - df_exec["entry_mnq"]) * np.where(df_exec["buy"] == "MNQ", 1, -1)
df_exec["px_side_mym"] = (df_exec["exit_mym"] - df_exec["entry_mym"]) * np.where(df_exec["buy"] == "MYM", 1, -1)
df_exec["pnl_mnq"] = df_exec["px_side_mnq"] * MNQ_LOT * 2
df_exec["pnl_mym"] = df_exec["px_side_mym"] * MYM_LOT * 0.5
df_exec["pnl_single"] = df_exec["pnl_mnq"] + df_exec["pnl_mym"] - 0.52 * (MNQ_LOT + MYM_LOT)
df_exec["pnl_cum"] = df_exec["pnl_single"].cumsum()
df_exec

,buy,sell,entry_t,entry_mnq,entry_mym,exit_mnq,exit_mym,exit_t,position,pnl_highest,pnl_lowest,hold_period,px_side_mnq,px_side_mym,pnl_mnq,pnl_mym,pnl_single,pnl_cum
0,MNQ,MYM,2022-02-21 02:35:00,14059.00,34217.0,14045.00,34193.0,2022-02-21 02:46:00,close,27.5,-19.0,0 days 00:11:00,-14.00,24.0,-28.0,36.0,5.92,5.92
1,MNQ,MYM,2022-02-21 02:55:00,14035.50,34197.0,14037.25,34189.0,2022-02-21 02:59:00,close,15.5,0.0,0 days 00:04:00,1.75,8.0,3.5,12.0,13.42,19.34
2,MYM,MNQ,2022-02-21 03:24:00,14011.50,34098.0,14016.75,34128.0,2022-02-21 03:26:00,close,7.5,0.0,0 days 00:02:00,-5.25,30.0,-10.5,45.0,32.42,51.76
3,MYM,MNQ,2022-02-21 03:44:00,14017.00,34112.0,13999.50,34090.0,2022-02-21 03:51:00,close,0.0,-12.5,0 days 00:07:00,17.50,-22.0,35.0,-33.0,-0.08,51.68
4,MNQ,MYM,2022-02-21 03:55:00,13985.75,34070.0,13973.75,34046.0,2022-02-21 04:02:00,close,8.5,-21.5,0 days 00:07:00,-12.00,24.0,-24.0,36.0,9.92,61.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,MYM,MNQ,2022-03-04 05:04:00,13926.50,33462.0,13922.00,33460.0,2022-03-04 05:12:00,close,16.0,0.0,0 days 00:08:00,4.50,-2.0,9.0,-3.0,3.92,580.52
131,MNQ,MYM,2022-03-04 05:29:00,13926.50,33491.0,13924.75,33475.0,2022-03-04 05:35:00,close,14.0,0.0,0 days 00:06:00,-1.75,16.0,-3.5,24.0,18.42,598.94
132,MYM,MNQ,2022-03-04 05:45:00,13923.50,33458.0,13929.25,33468.0,2022-03-04 05:57:00,close,24.5,0.0,0 days 00:12:00,-5.75,10.0,-11.5,15.0,1.42,600.36
133,MYM,MNQ,2022-03-04 06:05:00,13936.00,33458.0,13910.00,33410.0,2022-03-04 06:09:00,close,0.0,-24.5,0 days 00:04:00,26.00,-48.0,52.0,-72.0,-22.08,578.28


In [24]:
import plotly.express as px

fig = px.line(df_exec, x=df_exec.index, y="pnl_cum")
fig.show()